# Imports

In [1]:
from scikeras.wrappers import KerasRegressor
import tensorflow as tf
from tensorflow import keras
import pickle
from datetime import datetime as dt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import warnings
import calendar
import collections.abc
collections.Iterable = collections.abc.Iterable
warnings.filterwarnings("ignore")

2023-03-17 22:57:34.373971: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 22:57:35.037350: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-17 22:57:35.037405: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-17 22:57:35.037412: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-03-17 22:57:36.127049: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 22:57:36.178875: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 22:57:36.179039: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


# Data Ingestion and Preparation

In [3]:
train = pd.read_csv('train.csv', index_col='date')
train.index = pd.to_datetime(train.index)
train = pd.pivot_table(train, values='sales', index="date", columns=['store_nbr', 'family'], aggfunc=np.sum)
train.columns = [f"{a[0]}_{a[1].replace('/','_')}" for a in train.columns]
exog = pickle.load(open('exog.pkl', 'rb'))

In [4]:
exog

,dcoilwtico,National Holiday,Cotopaxi,Imbabura,Santa Elena,Santo Domingo de los Tsachilas,Ambato,Cayambe,Cuenca,El Carmen,...,Loja,Machala,Manta,Puyo,Quevedo,Quito,Riobamba,Salinas,Santo Domingo,Payday
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,93.205556,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2013-01-02,93.140000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2013-01-03,92.970000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2013-01-04,93.120000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2013-01-05,93.120000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-11,48.810000,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2017-08-12,48.810000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2017-08-13,47.590000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [5]:
emp = np.empty((train.shape[0], train.shape[1], 16))
for time_step in range(train.shape[0]):
    for feature in range(train.shape[1]):
        emp[time_step, feature] = [train.iloc[time_step+i, feature] if time_step+i<train.shape[0] else 0 for i in range(16)]
Y = emp

In [109]:
es = keras.callbacks.EarlyStopping(monitor='loss', mode='min', patience=10, min_delta=0.05)
X_train, X_test, y_train, y_test = train_test_split(exog, Y, test_size=0.10, random_state=5, shuffle=False)
X_train = X_train.values.reshape(1, X_train.shape[0], X_train.shape[1])
X_test = X_test.values.reshape(1, X_test.shape[0], X_test.shape[1])
y_train = y_train.reshape(1, y_train.shape[0], y_train.shape[1], y_train.shape[2])
y_test = y_test.reshape(1, y_test.shape[0], y_test.shape[1], y_test.shape[2])

In [110]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1, 1515, 26)
(1, 169, 26)
(1, 1515, 1782, 16)
(1, 169, 1782, 16)


## RNN model

In [111]:
rnn = keras.models.Sequential([
    keras.layers.SimpleRNN(100, return_sequences=True, input_shape=[None, X_train.shape[2]]),
    keras.layers.SimpleRNN(100, return_sequences=True),
    keras.layers.SimpleRNN(100),
    keras.layers.Dense(16)
])
rnn.compile(loss=keras.losses.MeanSquaredLogarithmicError(), optimizer=tf.keras.optimizers.Adam())
rnn.summary()
rnn.fit(X_train, y_train, epochs=200, callbacks=[es])
pred_rnn = np.round(abs(rnn.predict(X_test)))

Model: "sequential_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_120 (SimpleRNN)  (None, None, 100)         12700     
                                                                 
 simple_rnn_121 (SimpleRNN)  (None, None, 100)         20100     
                                                                 
 simple_rnn_122 (SimpleRNN)  (None, 100)               20100     
                                                                 
 dense_40 (Dense)            (None, 16)                1616      
                                                                 
Total params: 54,516
Trainable params: 54,516
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
1/1 [==============================] - 4s 4s/step - loss: 14.5474
Epoch 2/200
1/1 [==============================] - 2s 2s/step - loss: 13.7668
Epoch 3/200
1/1 [========

In [112]:
print(pred_rnn.shape)
pred_rnn

(1, 16)


array([[8., 8., 9., 2., 1., 1., 9., 7., 2., 8., 7., 8., 8., 8., 8., 8.]],
      dtype=float32)